# This file has moved
## New Name, New Location
PatentsView's Search API has become the PatentSearch API, and so the tutorial notebook has been renamed to match.  
You can find that resource at https://github.com/PatentsView/PatentsView-Code-Snippets/blob/master/07_PatentSearch_API_demo/PV%20PatentSearch%20API%20tutorial.ipynb  

Thank you for using PatentsView!